In [1]:
# import modules
import os
import shutil
from osgeo import gdal
import pandas as pd 
pd.set_option('display.max_rows', 10)
import geopandas as gpd

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyproj/__init__.py:78: UserWarning: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_LIB or with `pyproj.datadir.set_data_dir`.
  warnings.warn(str(err))


In [2]:
# track time
import time
start = time.time()
last = time.time()

# Generate area-of-interest for FUAs

In [3]:
# import csv containing lat/long for FUAs and create points gpd
fua_df = pd.read_csv('inputs/FUA-test.csv')
fua_gdf = gpd.GeoDataFrame(fua_df, geometry=gpd.points_from_xy(fua_df.longitude, fua_df.latitude), crs='EPSG:4326').to_crs(2193)

# create 70km buffer to get aoi
fua_gdf['aoi'] = fua_gdf.buffer(70000)

print(fua_gdf)
print() 
print('buffering FUAs complete')
print(time.time() - last, 'seconds')
last = time.time()
print() 


       Name  longitude  latitude                         geometry  \
0  Auckland   174.7645  -36.8508  POINT (1757311.536 5920225.659)   
1  Hamilton   175.2528  -37.7826  POINT (1798385.156 5815915.335)   
2     Taupo   176.0704  -38.6843  POINT (1867076.976 5713780.848)   

                                                 aoi  
0  POLYGON ((1827311.536 5920225.659, 1826974.467...  
1  POLYGON ((1868385.156 5815915.335, 1868048.087...  
2  POLYGON ((1937076.976 5713780.848, 1936739.907...  

buffering FUAs complete
0.14778780937194824 seconds



# Remove undevelopable land parcels

In [4]:
# import LCDB and return water bodies 
water_bodies = gpd.read_file('inputs/lcdb-v50-land-cover-database-version-50-mainland-new-zealand.shp')

water_bodies = water_bodies[water_bodies.Name_2018.isin(['Not land', 'Lake or Pond', 'River', 'Estuarine Open Water', 'Mangrove']) == False]

print(water_bodies)
print()
print('loading water_bodies complete')
print(time.time() - last, 'seconds')
last = time.time()
print()

                               Name_2018                         Name_2012  \
0           Herbaceous Saline Vegetation      Herbaceous Saline Vegetation   
1           Herbaceous Saline Vegetation      Herbaceous Saline Vegetation   
3           Herbaceous Saline Vegetation      Herbaceous Saline Vegetation   
5           Herbaceous Saline Vegetation      Herbaceous Saline Vegetation   
6           Herbaceous Saline Vegetation      Herbaceous Saline Vegetation   
...                                  ...                               ...   
511099           Low Producing Grassland           Low Producing Grassland   
511100                 Indigenous Forest                 Indigenous Forest   
511101                     Exotic Forest                     Exotic Forest   
511102  Herbaceous Freshwater Vegetation  Herbaceous Freshwater Vegetation   
511103   High Producing Exotic Grassland   High Producing Exotic Grassland   

                               Name_2008                       

# Calculate slope for developable land 

In [11]:
# filepath to dem data
dem = 'inputs/nz-dem-15m-2193.kea'

# create output dir for slope images and tmp dir for other files
slope_outputs = 'outputs/slope_images/'
tmp_dir = 'tmp/'
for folder in slope_outputs, tmp_dir:
    if not os.path.exists(folder):
            os.mkdir(folder)

# Clip water_bodies by each aoi
for index, row in fua_gdf.iterrows():
    # create tmp_dir for unrequired files and namePath for slope outputs
    tmpPath = tmp_dir + row['Name'] + '/'
    if not os.path.exists(tmpPath):
        os.mkdir(tmpPath)
    namePath = slope_outputs + row['Name']
    
    print(row)

    aoi_path = tmpPath + 'aoi.shp'
    developable_area = water_bodies.geometry.clip(row['aoi'])
    developable_area.to_file(aoi_path)

    developable_area_path = tmpPath + 'aoi.kea'

    # mask dem to get potential developable pixels within FUA aoi with gdal warp
    # output to memory
    gdal.Warp(destNameOrDestDS=developable_area_path,
    srcDSOrSrcDSTab=dem, format='kea', cutlineDSName=aoi_path, cutlineLayer='aoi', cropToCutline=True)

    print('masking dem complete')
    print(time.time() - last, 'seconds')
    last = time.time()
    print() 

    # Calculate slope using gdal DEMprocessing
    destPath = namePath + '-slope.kea'
    #srcDS = developable_area_path

    gdal.DEMProcessing(destName=destPath, srcDS=developable_area_path, processing='slope', format='kea', slopeFormat='degree')

    # Remove tmp folder
    shutil.rmtree(tmpPath)
    
    print('calculating slope complete')
    print(time.time() - last, 'seconds')
    last = time.time()
    print()
    
shutil.rmtree(tmp_dir)

print('Total processing time')
print(time.time() - start, 'seconds')
last = time.time()
print()

Name                                                  Auckland
longitude                                             174.7645
latitude                                              -36.8508
geometry           POINT (1757311.536241219 5920225.658568039)
aoi          POLYGON ((1827311.536241219 5920225.658568039,...
Name: 0, dtype: object


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


masking dem complete
933.9243009090424 seconds

calculating slope complete
4.483947992324829 seconds

Name                                                  Hamilton
longitude                                             175.2528
latitude                                              -37.7826
geometry           POINT (1798385.155720069 5815915.335158042)
aoi          POLYGON ((1868385.155720069 5815915.335158042,...
Name: 1, dtype: object


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


masking dem complete
312.25529408454895 seconds

calculating slope complete
7.519237279891968 seconds

Name                                                     Taupo
longitude                                             176.0704
latitude                                              -38.6843
geometry           POINT (1867076.975686512 5713780.847506042)
aoi          POLYGON ((1937076.975686512 5713780.847506042,...
Name: 2, dtype: object


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


masking dem complete
125.31549000740051 seconds

calculating slope complete
8.520750045776367 seconds

Total processing time
2367.2433009147644 seconds

